#52-文本预处理
##本节目录

*   [处理步骤](https://colab.research.google.com/drive/1oQH-UYj-DhB-DiUJ3nO2dEprKcNSD4F0#scrollTo=ltSVAkJi-SVF&line=6&uniqifier=1)
*   [读取数据集](https://colab.research.google.com/drive/1oQH-UYj-DhB-DiUJ3nO2dEprKcNSD4F0#scrollTo=BE1LFmRneFrp)
*   [词元化](https://colab.research.google.com/drive/1oQH-UYj-DhB-DiUJ3nO2dEprKcNSD4F0#scrollTo=FyvsoyYqeMuW&line=6&uniqifier=1)
*   [词表](https://colab.research.google.com/drive/1oQH-UYj-DhB-DiUJ3nO2dEprKcNSD4F0#scrollTo=aTVML0Pqeqb1&line=5&uniqifier=1)
*   [整合所有功能](https://colab.research.google.com/drive/1oQH-UYj-DhB-DiUJ3nO2dEprKcNSD4F0#scrollTo=cKrAEBT7gplL&line=6&uniqifier=1)




##启用GPU

#文本预处理
##处理步骤

---



1.   将文本作为字符串加载至内存
2.   将字符串拆分为词元（eg.单词和字符）
3.   建立词表，将拆分的词元映射到数字索引
4.   将文本转换为数字索引序列，方便模型操作



In [77]:
import collections
import re
from d2l import torch as d2l
import os

##读取数据集

In [78]:
#@save
# d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
#                                 '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():  #@save
    """将时间机器数据集加载到文本行的列表中"""
    with open(os.path.join('/home/peihuanni/d2l/data/', 'timemachine.txt'), 'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()
print(f'# 文本总行数: {len(lines)}')
print(lines[0])
print(lines[10])

# 文本总行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


##词元化


---



> 文本行列表中的每个文本序列被转化成以词元为单位的词元列表





In [79]:
#拆分数据为单词或字母
def tokenize(lines,token='word'):
    """将文本行拆分为单词或字符词元"""
    if token == 'word':
        return [line.split() for line in lines]
# ***************************************************************************
        # 也可以表述成下述：
        # lines_tokensized = []
        # for line in lines:
        #     lines_tokensized.append(line.split())
        # return lines_tokensized
        # 但要注意：不能写成这种形式：
        # for line in lines:
        #     line = line.split()
        # return line
        # 因为lines已经定义成了一维列表，这样在把虽然line.split()已经分割成了一维列表，但是最终还是会以字符串的形式存入
# ****************************************************************************
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('error: Unkown type:' + token)

tokens = tokenize(lines=lines, token='word')
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


## 词表


---


> 词表(vocabulary):是一个能让词元和数字索引互相映射的字典



> 语料(corpus):是文本中各个词元的出现频率的统计结果



> 根据词元出现频率分配数字索引，同时移除较少的词元降低词表的稀疏程度



> 特殊词元:
1.   未知词元<unk>
2.   填充词元<pad>
3.   序列开始词元<bos>
4.   序列结束词元<eos>









In [80]:
def count_corpus(tokens):
    """统计词元的频率"""
    #这里的tokens是1D列表或2D列表
    if len(tokens) == 0 or isinstance(tokens[0],list):        
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)
        #将词元列表展平成一个列表
        # ***********************************************************
        #另一种实现
        # corpus = []
        # for line in tokens:
        #     for token in line:
        #         corpus.append(token)
    # return collections.Counter(corpus) 
    # ***********************************************************

class Vocab:
    """文本词表"""
    def __init__(self,tokens=None,min_freq=0,reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        #按出现频率排序
        counter = count_corpus(tokens)
        # 展平成一维列表。形如：[]
        self._token_freqs = sorted(counter.items(),key=lambda x: x[1],reverse=True)
        #未知词元的索引为0
        self.idx_to_token = ['<unk>'] + reserved_tokens
        self.token_to_idx = {token: idx for idx,token in enumerate(self.idx_to_token)}
        # **************************************************************************************
        # self.token_to_idx = {}
        # for idx, token in enumerate(self.idx_to_token):
        #     self.token_to_idx.update({token: idx})
        # **************************************************************************************
        for token,freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self,tokens):
        if not isinstance(tokens,(list,tuple)):
            return self.token_to_idx.get(tokens,self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self,indices):
        if not isinstance(indices,(list,tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self): #未知词元的索引为0
        return 0

    @property
    def token_freqs(self):
        return self._token_freqs



In [81]:
test = [['The', 'Time', 'Machine,', 'by', 'H.', 'G.', 'Wells', '[1898]'],
[],
[],
[],
[],
['I'],
[],
[],
['The', 'Time', 'Traveller', '(for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him)'],
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us.', 'His', 'grey', 'eyes', 'shone', 'and'],
['twinkled,', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated.', 'The']]

result = count_corpus(test)

In [82]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


In [83]:
for i in [0,10]:
    print('token:',tokens[i])
    print('id:',vocab[tokens[i]])

token: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
id: [1, 19, 50, 40, 2183, 2184, 400]
token: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
id: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


##整合所有功能


---



1.   读取清洗后的文件
2.   词元化
3.   建立词表



In [84]:
def load_corpus_time_machine(max_tokens=-1):
    """返回数据集的词元索引列表和词表"""
    lines = read_time_machine()
    tokens = tokenize(lines,'char')
    vocab = Vocab(tokens)
    """将所有文本行展平到一个列表"""
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0 :
        corpus = corpus[:max_tokens]
    return corpus,vocab

corpus,vocab = load_corpus_time_machine()
len(corpus),len(vocab)

(170580, 28)